In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import random
import sklearn as sk
from time import time
import pickle

In [2]:
df=pd.read_csv("../data/MIXED1000patients_subsamples/1/CTGAN/brca_mRNA_patients_embedding_dim128_gen_dim256_dis_dim256_l2scale1e-06_batch_size500_epochs500.csv")

In [ ]:
df = pd.read_csv()
np.array()
G = nx.Graph()
f_arr = np.array(face_orig.readlines())
edgelist = []
for line in f_arr:
    i, j, time = line.split('\t')
    edgelist.append((int(i), int(j)))

G.add_edges_from(edgelist)